In [26]:
import pandas as pd
import glob
import collections
import numpy as np

In [20]:
files = glob.glob("../data/states/*.xlsx")

frames = {}

for f in files:
    key = f[-12:-8]
    frames[key] = pd.read_excel(f, engine='openpyxl', 
                             usecols="B,C,F,J,K,O,T")

# Used Columns: 
# B - STATE
# C - OCC_CODE; 
# F - TOT_EMP; 
# J - H_MEAN; 
# K - A_MEAN; 
# O - H_MEDIAN; 
# T - A_MEDIAN;

In [25]:
# sort frames by ascending order
frames = collections.OrderedDict(sorted(frames.items()))

In [27]:
def preprocess(df):
    df = df.replace(['*', '#', '**', '~'], np.NaN)  # drop non numberic values
    tmpDF = pd.DataFrame(columns=['OCC_CODE','SUB_CODE'])
    tmpDF[['OCC_CODE','SUB_CODE']] = df['OCC_CODE'].str.split('-', expand=True)
    df['OCC_CODE'] = tmpDF['OCC_CODE']
    return df

for key, value in frames.items():
    frames[key] = preprocess(value)

In [29]:
# aggregate by the same occ_code
def aggByOCC(df, column_name):
    new = df.groupby(['OCC_CODE', 'STATE']).agg({column_name: ['mean', 'min', 'max']})
    return new

# aggregate by mean annual wage 
agg_a_mean = {}

# aggregate by mean hourly wage 
agg_h_mean = {}

# aggregate by median annual wage 
agg_a_median = {}

# aggregate by median hourly wage 
agg_h_median = {}

for key, value in frames.items():
    agg_a_mean[key] = aggByOCC(value,'A_MEAN')
    agg_h_mean[key] = aggByOCC(value,'H_MEAN')
    agg_a_median[key] = aggByOCC(value,'A_MEDIAN')
    agg_h_median[key] = aggByOCC(value,'A_MEDIAN')

In [30]:
agg_a_mean

{'2012':                               A_MEAN                   
                                 mean      min       max
 OCC_CODE STATE                                         
 00       Alabama        39550.000000  39550.0   39550.0
          Alaska         52050.000000  52050.0   52050.0
          Arizona        43950.000000  43950.0   43950.0
          Arkansas       36850.000000  36850.0   36850.0
          California     52350.000000  52350.0   52350.0
 ...                              ...      ...       ...
 53       Virginia       44190.816327  20300.0  133090.0
          Washington     48950.714286  23490.0  137980.0
          West Virginia  41154.285714  18490.0   86270.0
          Wisconsin      40976.585366  18690.0   97360.0
          Wyoming        43174.242424  21490.0   72080.0
 
 [1239 rows x 3 columns],
 '2013':                               A_MEAN                   
                                 mean      min       max
 OCC_CODE STATE                             